# Tensorflow ImageNet FGM demo for Securing AI Lab deployment

This notebook contains a demonstration of the Fast Gradient Method (FGM) attack and Image Preprocessing defenses on the DGX workstation.

## Setup

**Note:** This demo is specifically for the NCCoE DGX Workstation with hostname `dgx-station-2`.

Port forwarding is required in order to run this demo.
The recommended port mapping is as follows:

- Map `localhost:30080` on laptop to `localhost:30080` on `dgx-station-2`
- Map `localhost:35000` on laptop to `localhost:35000` on `dgx-station-2`

A sample SSH config file that enables the above port forwarding is provided below,

> ⚠️ **Edits required**: replace `username` with your assigned username _on the NCCoE virtual machines_!

```conf
# vm hostname: jumphost001
Host nccoe-jumphost001
    Hostname 10.33.53.98
    User username
    Port 54131
    IdentityFile %d/.ssh/nccoe-vm

# vm hostname: dgx-station-2
Host nccoe-k8s-gpu002
    Hostname 192.168.1.28
    User username
    Port 22
    IdentityFile %d/.ssh/nccoe-vm
    ProxyJump nccoe-jumphost001
    LocalForward 30080 localhost:30080
    LocalForward 35000 localhost:35000
```

Now, connect to the NCCoE VPN and SSH into the DGX Workstation,

```bash
ssh nccoe-k8s-gpu002
```

Next, we import the necessary Python modules and ensure the proper environment variables are set so that all the code blocks will work as expected,

In [151]:
# Import packages from the Python standard library
import os
import pprint
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "howard"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device (see above instructions for details)
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    f"http://{HOST_DOCKER_INTERNAL}:30080"
    if os.getenv("IS_JUPYTER_SERVICE")
    else "http://localhost:30080"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = "http://localhost:35000"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_imagenet_fgm_defense"

# Set MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
if os.getenv("MLFLOW_TRACKING_URI") is None:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

Check that the Makefile works in your environment by executing the `bash` code block below,

In [152]:
%%bash

# Running this will just list the available rules defined in the demo's Makefile.
make

Available rules:

clean               Remove temporary files 
data                Download and prepare MNIST dataset 
demo                Launch the demo 
initdb              Initialize the RESTful API database 
jupyter             Launch the jupyter lab service 
plugins             Synchronize plugins to S3 
services            Launch the Testbed services 
teardown            Stop and remove the Testbed service containers 
workflows           Create workflows tarball 


spatial_smoothingThe ImageNet dataset needed for the demo is available in all worker containers at the path `/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/`.
The following directores contain labeled JPEG images:

    train  
    val-sorted  
    val-sorted-1000
    val-sorted-5000
    val-sorted-10000  
    
Both train and val-sorted contain a full set of labeled training and test images. val-sorted-# directories hold a random partion of labeled images, with 1000, 5000, or 10000 images in total. 

Each of the directories listed follows the general subfolder structure below:

    ImageNet
    │ 
    │ 
    ├── train
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}
    │ 
    └── val-sorted
    │ 
    │   ├── {class 0 directory}
    │   ├── {class 1 directory}
    │   ├── ...
    │   ├── ...
    │   ├── ...
    │   └── {class 999 directory}

The subfolders under each directory are the classification labels for the images in the dataset.
This folder structure is a standardized way to encode the label information and many libraries can make use of it, including the Tensorflow library that we are using for this particular demo.

## Submit and run jobs

The jobs that we will be running are implemented in the Python source files under `src/`, which will be executed using the entrypoints defined in the `MLproject` file.
To get this information into the architecture, we need to package those files up into an archive and upload it to the lab API.
For convenience, the `Makefile` provides a rule for creating the archive file, just run `make workflows`,

In [153]:
%%bash

# Create the workflows.tar.gz file
make workflows

make: Nothing to be done for 'workflows'.


To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the lab's RESTful API using the HTTP protocol.
We connect using the client below,

In [154]:
restapi_client = utils.SecuringAIClient(address=RESTAPI_API_BASE)

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment named `"howard_imagenet_fgm_defense"` exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [155]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

{'experimentId': 8,
 'name': 'howard_imagenet_fgm_defense',
 'lastModified': '2020-10-29T04:14:12.391042',
 'createdOn': '2020-10-29T04:14:12.391042'}

# FGM Model Setup and Attack

For this section of the demo we will first start by loading in the respective baseline ResNet50 model and evaluating its performance on a subset of the ImageNet validation data.

In [227]:
response_init_model = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="init_model",
    entry_point_kwargs=" ".join([
        "-P batch_size=20",
        "-P model_architecture=resnet50",
        f"-P register_model_name={EXPERIMENT_NAME}_pretrained_resnet50",
        "-P imagenet_preprocessing=true",
        "-P image_size=224,224,3",
        "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
    ]),
    queue="tensorflow_gpu",
    timeout="1h",
)

print("Model initialization and ImageNet evaluation job submitted.")
print("")
pprint.pprint(response_init_model)


Model initialization and ImageNet evaluation job submitted.

{'createdOn': '2021-03-26T07:50:36.297706',
 'dependsOn': None,
 'entryPoint': 'init_model',
 'entryPointKwargs': '-P batch_size=20 -P model_architecture=resnet50 -P '
                     'register_model_name=howard_imagenet_fgm_defense_pretrained_resnet50 '
                     '-P imagenet_preprocessing=true -P image_size=224,224,3 '
                     '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000',
 'experimentId': 8,
 'jobId': '57734dfd-c021-4a44-b479-e35fffb0e890',
 'lastModified': '2021-03-26T07:50:36.297706',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '1h',
 'workflowUri': 's3://workflow/3aa55ed812cc43bf83f669b965c0449e/workflows.tar.gz'}


Now that we have our baseline model loaded and stored, next we will apply the fast-gradient method (FGM) evasion attack on the model to generate adversarial images.
Then, after we have the adversarial images, we will reapply the attack with spatial smoothing defense enabled.

We will then test the model under both sets of images (FGM attack, FGM attack + preprocessing defense) and apply standard machine learning accuracy metrics on the inference results. 

This will give us a general sense of the effectivness and robustness of the FGM evasion attack with regards to a particular preprocessing defense.


In [343]:
def mlflow_run_id_is_not_known(response):
    return response["mlflowRunId"] is None and response["status"] not in [
        "failed",
        "finished",
    ]

response_fgm_resnet50_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="fgm",
    entry_point_kwargs=" ".join(
        [
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
            "-P batch_size=20",
            "-P eps_step=0.1",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
            "-P eps=10"
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_init_model['jobId']
)

print("FGM attack (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack)
print("")

# Wait for FGM attack to complete.Then evaluate results.
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack):
    time.sleep(1)
    response_fgm_resnet50_attack = restapi_client.get_job_by_id(response_fgm_resnet50_attack["jobId"])

FGM attack (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:03:10.032827',
 'dependsOn': '57734dfd-c021-4a44-b479-e35fffb0e890',
 'entryPoint': 'fgm',
 'entryPointKwargs': '-P '
                     'model_name=howard_imagenet_fgm_defense_pretrained_resnet50 '
                     '-P model_version=None -P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000 '
                     '-P batch_size=20 -P eps_step=0.1 -P '
                     'imagenet_preprocessing=true -P image_size=224,224,3 -P '
                     'eps=0.1',
 'experimentId': 8,
 'jobId': '4982fe00-7d66-447c-a317-579c56fea3ca',
 'lastModified': '2021-03-27T08:03:10.032827',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/7f79d6ee31974e278ccb9653260381e7/workflows.tar.gz'}



In [344]:
response_evaluate_fgm_resnet50_attack = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=testing_adversarial_fgm.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack)
print("")

FGM evaluation (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:03:15.173568',
 'dependsOn': '4982fe00-7d66-447c-a317-579c56fea3ca',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=b2f3b9e0d12c4da19e76cfb178fb1a5f -P '
                     'model_name=howard_imagenet_fgm_defense_pretrained_resnet50 '
                     '-P model_version=None -P batch_size=20 -P '
                     'adv_tar_name=testing_adversarial_fgm.tar.gz -P '
                     'adv_data_dir=adv_testing -P imagenet_preprocessing=true '
                     '-P image_size=224,224,3',
 'experimentId': 8,
 'jobId': '936d78bd-8e9c-4572-bbbc-d629f651cb9e',
 'lastModified': '2021-03-27T08:03:15.173568',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/25a5c5865b57462b8b766d21864228dd/workflows.tar.gz'}



# Image Preprocessing Defenses
Now we will apply our defenses over the fgm-attacked images.

In [345]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="spatial_smoothing",
    entry_point_kwargs=" ".join(
        [
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
            "-P image_size=224,224,3",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM spatial smoothing defense (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

FGM spatial smoothing defense (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:03:15.232311',
 'dependsOn': '4982fe00-7d66-447c-a317-579c56fea3ca',
 'entryPoint': 'spatial_smoothing',
 'entryPointKwargs': '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000 '
                     '-P batch_size=20 -P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=b2f3b9e0d12c4da19e76cfb178fb1a5f -P '
                     'dataset_tar_name=testing_adversarial_fgm.tar.gz -P '
                     'dataset_name=adv_testing -P image_size=224,224,3',
 'experimentId': 8,
 'jobId': '22a7a62c-c685-493b-8f38-dc89f27f960f',
 'lastModified': '2021-03-27T08:03:15.232311',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/44ee47abc14d4723b960056dc630c917/workflows.tar.gz'}



In [346]:
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=spatial_smoothing_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")

FGM evaluation (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:05:16.259931',
 'dependsOn': '22a7a62c-c685-493b-8f38-dc89f27f960f',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=be06528445cb4326a1a725789de00026 -P '
                     'model_name=howard_imagenet_fgm_defense_pretrained_resnet50 '
                     '-P model_version=None -P batch_size=20 -P '
                     'adv_tar_name=spatial_smoothing_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing -P imagenet_preprocessing=true '
                     '-P image_size=224,224,3',
 'experimentId': 8,
 'jobId': 'edf4468b-dd2f-4df7-a300-bc15706d0b14',
 'lastModified': '2021-03-27T08:05:16.259931',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/a5ed1b971d91483cb30828378755c91e/workflows.tar.gz'}



In [348]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="gaussian_augmentation",
    entry_point_kwargs=" ".join(
        [
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
            "-P image_size=224,224,3",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM gaussian augmentation defense (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

FGM gaussian augmentation defense (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:05:16.355603',
 'dependsOn': '4982fe00-7d66-447c-a317-579c56fea3ca',
 'entryPoint': 'gaussian_augmentation',
 'entryPointKwargs': '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000 '
                     '-P batch_size=20 -P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=b2f3b9e0d12c4da19e76cfb178fb1a5f -P '
                     'dataset_tar_name=testing_adversarial_fgm.tar.gz -P '
                     'dataset_name=adv_testing -P image_size=224,224,3',
 'experimentId': 8,
 'jobId': '667fffcb-c95c-4850-93f6-cdf3911d20a9',
 'lastModified': '2021-03-27T08:05:16.355603',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/a549d3dd8da0496ea94dad6f38ba6923/workflows.tar.gz'}



In [349]:
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=gaussian_augmentation_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")

FGM evaluation (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:05:24.548698',
 'dependsOn': '667fffcb-c95c-4850-93f6-cdf3911d20a9',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=7f145640f9f9450b96ec2d50aa7885ef -P '
                     'model_name=howard_imagenet_fgm_defense_pretrained_resnet50 '
                     '-P model_version=None -P batch_size=20 -P '
                     'adv_tar_name=gaussian_augmentation_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing -P imagenet_preprocessing=true '
                     '-P image_size=224,224,3',
 'experimentId': 8,
 'jobId': '8a4a6090-733b-44cb-a935-bc57da54ee69',
 'lastModified': '2021-03-27T08:05:24.548698',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/1ef28ffbfd2b497ab74bb6cbabcaa297/workflows.tar.gz'}



In [350]:
response_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="jpeg_compression",
    entry_point_kwargs=" ".join(
        [
            "-P data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000",
            "-P batch_size=20",
            "-P load_dataset_from_mlruns=true",
            f"-P dataset_run_id={response_fgm_resnet50_attack['mlflowRunId']}",
            "-P dataset_tar_name=testing_adversarial_fgm.tar.gz",
            "-P dataset_name=adv_testing",
            "-P image_size=224,224,3",
            
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack["jobId"],
)

print("FGM JPEG compression defense (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_fgm_resnet50_attack_def)
print("")

FGM JPEG compression defense (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:05:24.608562',
 'dependsOn': '4982fe00-7d66-447c-a317-579c56fea3ca',
 'entryPoint': 'jpeg_compression',
 'entryPointKwargs': '-P '
                     'data_dir=/nfs/data/ImageNet-Kaggle-2017/images/ILSVRC/Data/CLS-LOC/val-sorted-1000 '
                     '-P batch_size=20 -P load_dataset_from_mlruns=true -P '
                     'dataset_run_id=b2f3b9e0d12c4da19e76cfb178fb1a5f -P '
                     'dataset_tar_name=testing_adversarial_fgm.tar.gz -P '
                     'dataset_name=adv_testing -P image_size=224,224,3',
 'experimentId': 8,
 'jobId': '57745614-4069-408e-a1b6-2b78bde90f02',
 'lastModified': '2021-03-27T08:05:24.608562',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/852540f318184726a4e8405d42c2ff31/workflows.tar.gz'}



In [351]:
while mlflow_run_id_is_not_known(response_fgm_resnet50_attack_def):
    time.sleep(1)
    response_fgm_resnet50_attack_def = restapi_client.get_job_by_id(response_fgm_resnet50_attack_def["jobId"])
    
    
response_evaluate_fgm_resnet50_attack_def = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    entry_point="infer",
    entry_point_kwargs=" ".join(
        [
            f"-P run_id={response_fgm_resnet50_attack_def['mlflowRunId']}",
            f"-P model_name={EXPERIMENT_NAME}_pretrained_resnet50",
            "-P model_version=None",
            "-P batch_size=20",
            "-P adv_tar_name=jpeg_compression_dataset.tar.gz",
            "-P adv_data_dir=adv_testing",
            "-P imagenet_preprocessing=true",
            "-P image_size=224,224,3",
        ]
    ),
    queue="tensorflow_gpu",
    depends_on=response_fgm_resnet50_attack_def["jobId"],
)

print("FGM evaluation (ResNet50 architecture) job submitted")
print("")
pprint.pprint(response_evaluate_fgm_resnet50_attack_def)
print("")

FGM evaluation (ResNet50 architecture) job submitted

{'createdOn': '2021-03-27T08:05:44.999203',
 'dependsOn': '57745614-4069-408e-a1b6-2b78bde90f02',
 'entryPoint': 'infer',
 'entryPointKwargs': '-P run_id=7bfa1fa540cf41859c71cbe8c53d6803 -P '
                     'model_name=howard_imagenet_fgm_defense_pretrained_resnet50 '
                     '-P model_version=None -P batch_size=20 -P '
                     'adv_tar_name=jpeg_compression_dataset.tar.gz -P '
                     'adv_data_dir=adv_testing -P imagenet_preprocessing=true '
                     '-P image_size=224,224,3',
 'experimentId': 8,
 'jobId': '9fd6409e-6acf-4b83-a57e-ca391d5e0d3a',
 'lastModified': '2021-03-27T08:05:44.999203',
 'mlflowRunId': None,
 'queueId': 2,
 'status': 'queued',
 'timeout': '24h',
 'workflowUri': 's3://workflow/dffcaffd674d44fea8e7df9b3c463003/workflows.tar.gz'}



## Querying the MLFlow Tracking Service

Currently the lab API can only be used to register experiments and start jobs, so if users wish to extract their results programmatically, they can use the `MlflowClient()` class from the `mlflow` Python package to connect and query their results.
Since we captured the run ids generated by MLFlow, we can easily retrieve the data logged about one of our jobs and inspect the results.
To start the client, we simply need to run,

In [ ]:
mlflow_client = MlflowClient()

The client uses the environment variable `MLFLOW_TRACKING_URI` to figure out how to connect to the MLFlow Tracking Service, which we configured near the top of this notebook.
To query the results of one of our runs, we just need to pass the run id to the client's `get_run()` method.
As an example, let's query the run results for the FGM attack applied to the LeNet-5 architecture,

In [ ]:
fgm_run = mlflow_client.get_run(response_evaluate_fgm_resnet50_attack_def["mlflowRunId"])

If the request completed successfully, we should now be able to query data collected during the run.
For example, to review the collected metrics, we just use,

In [ ]:
pprint.pprint(fgm_run.data.metrics)

To review the run's parameters, we use,

In [ ]:
pprint.pprint(fgm_run.data.params)

To review the run's tags, we use,

In [ ]:
pprint.pprint(fgm_run.data.tags)

There are many things you can query using the MLFlow client.
[The MLFlow documentation gives a full overview of the methods that are available](https://www.mlflow.org/docs/latest/python_api/mlflow.tracking.html#mlflow.tracking.MlflowClient).